In [1]:
!pip install fredapi

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

from fredapi import Fred

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("FRED")

# Create the Fred Object

In [3]:
fred = Fred(api_key=fred_key)

NameError: name 'fred_key' is not defined

# Search for economic data

In [ ]:
fred.search('S&P')

In [ ]:
sp_search = fred.search('S&P',order_by='popularity')

In [ ]:
sp_search.shape

In [ ]:
sp_search.head(10)

# Pull Raw Data and Plot

In [ ]:
sp500 = fred.get_series(series_id='SP500')

In [ ]:
sp500.plot(figsize=(10,5),title='S&P 500',lw=2);

# Pull and Joing Multiple Data Series

In [ ]:
unemp_result = fred.search('unemployment')

In [ ]:
unrate = fred.get_series('UNRATE')

In [ ]:
unrate.plot(figsize=(12,5),title='Unemployment rate',lw=2);

In [ ]:
unemp_result

In [ ]:
unemp_df = fred.search('unemployment rate state', filter=('frequency', 'Monthly'))
unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemp_df = unemp_df.loc[unemp_df['title'].str.contains('Unemployment Rate')]

In [ ]:
unemp_df

In [ ]:
unemp_df.index

In [ ]:
len(unemp_df.index)

In [ ]:
unemp_df_list_of_columns = unemp_df.index.tolist()
print(unemp_df_list_of_columns)

In [ ]:
list_to_drop = [elem for elem in unemp_df_list_of_columns if len(elem) > 4]

In [ ]:
#This cell usually takes long to run, it will clean the dataframe and come up with a new one with only the states meta data

all_results = []

for myid in unemp_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
#Drop all the columns not corresponding to states in this line
unemp_states = pd.concat(all_results, axis=1).drop(list_to_drop, axis=1)

In [ ]:
unemp_states

In [ ]:
unemp_states.isna().sum(axis=1)

In [ ]:
unemp_states.isna().sum(axis=1).plot()

In [ ]:
unemp_states = unemp_states.dropna()

In [ ]:
unemp_df['title']=='Unemployment Rate in California'

In [ ]:
id_to_state = unemp_df['title'].str.replace('Unemployment Rate in', '').str.strip().to_dict()
unemp_states.columns = unemp_states.columns.str.strip()
unemp_states.columns = [id_to_state[c] if c in id_to_state else c for c in unemp_states.columns]
unemp_states = unemp_states.loc[:, ~unemp_states.columns.duplicated()]

In [ ]:
print(len(id_to_state))

In [ ]:
# plot States Unemployment Rate
px.line(unemp_states)

## Pull May 2020 Unemployment Rate Per State

In [ ]:
print(unemp_states.loc[unemp_states.index == '2020-05-01'].T)

In [ ]:
ax = unemp_states.loc[unemp_states.index == '2020-05-01'].T.sort_values('2020-05-01').plot(kind='barh',figsize=(12,12), width=1, edgecolor='black',
                                                                                      title='Unemployment Rate by State, May 2020');
ax.legend().remove()
plt.show()

In [ ]:
#print(unemp_states.loc[unemp_states.index].T)

# Pull Participation Rate

In [ ]:
part_df = fred.search('participation rate state', filter=('frequency', 'Monthly'))
part_df = part_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')

In [ ]:
part_id_to_state = part_df['title'].str.replace('Labor Force Participation Rate for ','').to_dict()

In [ ]:
to_drop = {key: value for key, value in part_id_to_state.items() if not key.startswith("LBS")}
to_drop_list = list(to_drop.keys())
print(to_drop_list)

In [ ]:
all_results = []

for myid in part_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
part_states = pd.concat(all_results, axis=1).drop(to_drop_list, axis=1)
part_states.columns = [part_id_to_state[c] for c in part_states.columns]

In [ ]:
part_states = part_states.dropna()

# Plot Unumployment vs Participation

In [ ]:
# Fix DC
unemp_states = unemp_states.rename(columns={'the District of Columbia':'District of Columbia'})

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
unemp_states.query('index >= 2020 and index < 2022')['New York'].plot(ax=ax, label='Unemployment');
#part_states.query('index >= 2020 and index < 2022')['New York'].plot(ax=ax2, label='Participation', color=color_pal[1]);

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
#unemp_states.query('index >= 2020 and index < 2022')['New York'].plot(ax=ax, label='Unemployment');
part_states.query('index >= 2020 and index < 2022')['New York'].plot(ax=ax2, label='Participation', color=color_pal[1]);

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(20,20), sharex=True, sharey=True)
axs = axs.flatten()

i=0
for state in unemp_states.columns:
    if state in ['District of Columbia', 'Puerto Rico']:
        continue
    ax2 = axs[i].twinx()
    unemp_states.query('index >= 2020 and index < 2022')[state].plot(ax=axs[i], label='Unemployment');
    part_states.query('index >= 2020 and index < 2022')[state].plot(ax=ax2, label='Participation', color=color_pal[1]);
    ax2.grid(False)
    axs[i].set_title(state, fontsize=8)
    i+= 1
plt.tight_layout()
plt.show()

In [ ]:
# Merge the two DataFrames column-wise
merged_df = pd.concat([part_states, unemp_states], axis=1, keys=['participation', 'unemployment'])

# Print the first 5 rows of the merged DataFrame
#print(merged_df.head())

In [ ]:
state = 'California'
start_date = '2020-01-01'
end_date = '2021-12-31'

unemp_states.loc[start_date:end_date, state].plot(label='Unemployment Rate')
part_states.loc[start_date:end_date, state].plot(label='Participation Rate')
plt.title(f'Unemployment and Participation Rates for {state} ({start_date} to {end_date})')
plt.legend()
plt.show()

In [ ]:
state = 'New York'
start_date = '2020-01-01'
end_date = '2021-12-31'

unemp_states.loc[start_date:end_date, state].plot(label='Unemployment Rate')
part_states.loc[start_date:end_date, state].plot(label='Participation Rate')
plt.title(f'Unemployment and Participation Rates for {state} ({start_date} to {end_date})')
plt.legend()
plt.show()